# Sextupole - Comparing to Bmad

In [ ]:
import numpy as np
from pmd_beamphysics import single_particle
from pmd_beamphysics.units import mec2
from pytao import Tao

import impact.z as IZ
from impact.tests.z.conftest import bmad_files
from impact.z.interfaces.bmad import plot_impactz_and_tao_stats, track_tao

## Initial Particles

In [ ]:
energy = 10e6
pz = np.sqrt(energy**2 - mec2**2)
P0 = single_particle(x=1e-3, pz=pz)

## Tao

In [ ]:
!cat $bmad_files/sextupole.bmad

In [ ]:
tao = Tao(lattice_file=bmad_files / "sextupole.bmad", plot="mpl")
tao.plot("beta", include_layout=False)

In [ ]:
Ptao = track_tao(tao, P0)

## Impact-Z

In [ ]:
I = IZ.ImpactZ.from_tao(tao, initial_particles=P0)
I.input

In [ ]:
output = I.run()

In [ ]:
P = I.output.particles["final_particles"]

# Compare

In [ ]:
plot_impactz_and_tao_stats(I, tao)

In [ ]:
P.data

In [ ]:
Ptao.data